# **Requirements**

In [ ]:
%pip install vizdoom
%pip install sample-factory

# **Functionality**

In [ ]:
import functools

from sample_factory.algo.utils.context import global_model_factory
from sample_factory.cfg.arguments import parse_full_cfg, parse_sf_args
from sample_factory.envs.env_utils import register_env
from sample_factory.train import run_rl

from sf_examples.vizdoom.doom.doom_model import make_vizdoom_encoder
from sf_examples.vizdoom.doom.doom_params import add_doom_env_args, doom_override_defaults
from sf_examples.vizdoom.doom.doom_utils import DOOM_ENVS, make_doom_env_from_spec


# Registers all the ViZDoom environments
def register_vizdoom_envs():
    for env_spec in DOOM_ENVS:
        make_env_func = functools.partial(make_doom_env_from_spec, env_spec)
        register_env(env_spec.name, make_env_func)

# Sample Factory allows the registration of a custom Neural Network architecture
# See https://github.com/alex-petrenko/sample-factory/blob/master/sf_examples/vizdoom/doom/doom_model.py for more details
def register_vizdoom_models():
    global_model_factory().register_encoder_factory(make_vizdoom_encoder)


def register_vizdoom_components():
    register_vizdoom_envs()
    register_vizdoom_models()

# parse the command line args and create a config
def parse_vizdoom_cfg(argv=None, evaluation=False):
    parser, _ = parse_sf_args(argv=argv, evaluation=evaluation)
    # parameters specific to Doom envs
    add_doom_env_args(parser)
    # override Doom default values for algo parameters
    doom_override_defaults(parser)
    # second parsing pass yields the final configuration
    final_cfg = parse_full_cfg(parser, argv)
    return final_cfg

# **Training**

In [ ]:
## Start the training, this should take around  minutes
register_vizdoom_components()

# other scenarios include "doom_basic", "doom_two_colors_easy", "doom_dm", "doom_dwango5", "doom_my_way_home", "doom_deadly_corridor", "doom_defend_the_center", "doom_defend_the_line"

env = "doom_deathmatch_bots"
cfg = parse_vizdoom_cfg(argv=[f"--env={env}",
                              "--algo=APPO",
                              "--seed=0",
                              "--num_workers=16",
                              "--num_envs_per_worker=24",
                              "--batch_size=2048",
                              "--recurrence=32",
                              "--gamma=0.995",
                              "--res_w=128",
                              "--res_h=72",
                              "--train_for_env_steps=10000000",
                              "--train_for_seconds=36000",
                              "--use_rnn=True",
                              "--rnn_type=lstm",
                              "--nonlinearity=relu",
                              "--env_frameskip=2"
                             ])

status = run_rl(cfg)

[2024-08-01 10:28:50,463][00281] Environment doom_basic already registered, overwriting...
[2024-08-01 10:28:50,466][00281] Environment doom_two_colors_easy already registered, overwriting...
[2024-08-01 10:28:50,468][00281] Environment doom_two_colors_hard already registered, overwriting...
[2024-08-01 10:28:50,472][00281] Environment doom_dm already registered, overwriting...
[2024-08-01 10:28:50,473][00281] Environment doom_dwango5 already registered, overwriting...
[2024-08-01 10:28:50,475][00281] Environment doom_my_way_home_flat_actions already registered, overwriting...
[2024-08-01 10:28:50,476][00281] Environment doom_defend_the_center_flat_actions already registered, overwriting...
[2024-08-01 10:28:50,477][00281] Environment doom_my_way_home already registered, overwriting...
[2024-08-01 10:28:50,478][00281] Environment doom_deadly_corridor already registered, overwriting...
[2024-08-01 10:28:50,482][00281] Environment doom_defend_the_center already registered, overwriting...

# **Eval**

In [ ]:
from sample_factory.enjoy import enjoy
register_vizdoom_components()
env = "doom_deathmatch_bots"
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=10"], evaluation=True)
status = enjoy(cfg)

[2024-08-01 10:09:21,418][00281] Environment doom_basic already registered, overwriting...
[2024-08-01 10:09:21,420][00281] Environment doom_two_colors_easy already registered, overwriting...
[2024-08-01 10:09:21,422][00281] Environment doom_two_colors_hard already registered, overwriting...
[2024-08-01 10:09:21,424][00281] Environment doom_dm already registered, overwriting...
[2024-08-01 10:09:21,426][00281] Environment doom_dwango5 already registered, overwriting...
[2024-08-01 10:09:21,428][00281] Environment doom_my_way_home_flat_actions already registered, overwriting...
[2024-08-01 10:09:21,430][00281] Environment doom_defend_the_center_flat_actions already registered, overwriting...
[2024-08-01 10:09:21,431][00281] Environment doom_my_way_home already registered, overwriting...
[2024-08-01 10:09:21,432][00281] Environment doom_deadly_corridor already registered, overwriting...
[2024-08-01 10:09:21,434][00281] Environment doom_defend_the_center already registered, overwriting...

# **Viz**

In [ ]:
from base64 import b64encode
from IPython.display import HTML

mp4 = open('train_dir/default_experiment/replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=640 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# **Save**

In [ ]:
from sample_factory.enjoy import enjoy

hf_username = "Mojitrk" # insert your HuggingFace username here

cfg = parse_vizdoom_cfg(argv=[f"--env={env}",
                              "--num_workers=1",
                              "--save_video",
                              "--no_render",
                              "--max_num_episodes=10",
                              "--max_num_frames=100000",
                              "--push_to_hub",
                              f"--hf_repository={hf_username}/doom_deathmatch_bots"], evaluation=True)
status = enjoy(cfg)